## Merge surprisal of all sentences with df:

In [3]:
import pandas as pd

path = "celer/data_v2.0/sent_ia.tsv"
# full_data = pd.read_csv('sent_ia.tsv', sep='\t')
full_data = pd.read_csv(path, sep=r'\t+', engine='python')

import pickle
surp_dict = pickle.load(open("surprisal_data/surprisal_results.pkl", "rb"))

In [21]:
full_data.keys()

Index(['DATA_FILE', 'list', 'dataset_version', 'trial', 'shared_text',
       'sentenceid', 'IA_ID', 'IA_LABEL', 'IA_LEFT', 'IA_RIGHT',
       'IA_FIRST_FIXATION_X', 'IA_DWELL_TIME', 'IA_DWELL_TIME_%',
       'IA_FIRST_FIXATION_DURATION', 'IA_FIRST_FIXATION_INDEX',
       'IA_FIRST_FIXATION_PREVIOUS_FIX_IA',
       'IA_FIRST_FIXATION_PREVIOUS_IAREAS',
       'IA_FIRST_FIXATION_VISITED_IA_COUNT', 'IA_FIRST_FIXATION_RUN_INDEX',
       'IA_FIRST_FIX_PROGRESSIVE', 'IA_FIRST_SACCADE_ANGLE',
       'IA_FIRST_RUN_LANDING_POSITION', 'IA_FIRST_RUN_DWELL_TIME',
       'IA_FIRST_RUN_FIXATION_COUNT', 'IA_FIRST_RUN_LAUNCH_SITE',
       'IA_REGRESSION_PATH_DURATION', 'IA_REGRESSION_IN',
       'IA_REGRESSION_IN_COUNT', 'IA_REGRESSION_OUT', 'IA_REGRESSION_OUT_FULL',
       'IA_REGRESSION_OUT_COUNT', 'IA_REGRESSION_OUT_FULL_COUNT',
       'IA_FIXATION_COUNT', 'IA_RUN_COUNT', 'IA_SKIP', 'IP_START_TIME',
       'IP_END_TIME', 'EYE_USED', 'TRIAL_FIXATION_COUNT', 'TRIAL_IA_COUNT',
       'TRIAL_DWELL_TIME

In [4]:
# we might only want to keep the sentences that are not shared:
data = full_data[full_data['shared_text'] == 0]
# remove if sentence is nan:
data = data.dropna(subset=['sentence'])

In [46]:
from tqdm import tqdm

idx = 0
save_to = 'surprisal_data/merged_data.tsv'
freq = "subtlex_Frequency"
surp1 = 'EleutherAI/pythia-70m_Surprisal'
surp2 = 'EleutherAI/pythia-160m_Surprisal'
surp3 = 'EleutherAI/pythia-410m_Surprisal'
bad_rows = [154, 283, 366, 377, 943, 1167, 1176, 1188, 1192, 1218, 1228, 1252, 1293,
            1300, 1328, 1330, 1390, 1425, 1475, 1632, 1810]

# Track faulty rows:
bad_sentences = []
count_errors = 0

with open(save_to, 'w') as f:
    header = list(full_data.columns) + [surp1, surp2, surp3]
    f.write('\t'.join(header) + '\n')

    for i, row in tqdm(data.iterrows()):

        real_word = row['IA_LABEL']
        sentence = row['sentence']

        if sentence in surp_dict:
            sentence_surp = surp_dict[sentence]
        elif sentence.replace('"', '') in surp_dict: # fix formatting issues
            sentence_surp = surp_dict[sentence.replace('"', '')]
        else:
            bad_sentences.append(sentence)  
            count_errors += 1
            # print(f'Error: sentence {i} not found in surp_dict')
            continue

        a_row = sentence_surp.loc[sentence_surp['Word'] == real_word]
        if a_row.empty:
            try:
                real_word = real_word.replace('"', '')
                a_row = sentence_surp.loc[sentence_surp['Word'].str.replace('"', '') == real_word]

                if a_row.empty:
                    count_errors += 1
                    # print(f'Error: {real_word} not found in sentence {i}')
                    continue
            
            except Exception as e:
                count_errors += 1
                # print(f'Error: {real_word} not found in sentence {i}')
                continue
        
        if i in bad_rows:
            count_errors += 1
            continue
        
        # All checks passed, write to file:
        else:
            row_data = row.tolist()
            row_data += [a_row[freq].values[0], a_row[surp1].values[0], a_row[surp2].values[0], a_row[surp3].values[0]]
            f.write('\t'.join(map(str, row_data)) + '\n')

print(f'Number of errors: {count_errors}')

320221it [01:10, 4573.73it/s]

Number of errors: 879


In [8]:
import pandas as pd

# read with no index col:
out_df = pd.read_csv("surp_data/merged_data.tsv", sep='\t', index_col=False)
out_df.shape

/var/folders/b1/6p4sk8f166d1j8xr27c9vfxm0000gp/T/ipykernel_37177/3106504020.py:4: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  out_df = pd.read_csv("surp_data/merge_output3.tsv", sep='\t', index_col=False)


(319342, 60)

## Short example of how the data is expected to look:

In [12]:
out_df.head(50)

,DATA_FILE,list,dataset_version,trial,shared_text,sentenceid,IA_ID,IA_LABEL,IA_LEFT,IA_RIGHT,...,FREQ_SUBTLEX,OOV_WEB,FREQ_WEB,SURP_KENLM,SURP_LSTM,SURP_GPT2,WORD_LEN,EleutherAI/pythia-70m_Surprisal,EleutherAI/pythia-160m_Surprisal,EleutherAI/pythia-410m_Surprisal
0,cl2200.edf,154,1,2,0,en-universal-train.conllu 225,1,Real,58,106,...,11.104360,0.0,10.948086,12.813140,37.069229,11.659881,4.0,11.104360,7.862239,8.860493
1,cl2200.edf,154,1,2,0,en-universal-train.conllu 225,2,"good,",106,166,...,8.544974,0.0,10.650781,9.618012,11.446278,16.230761,4.0,8.544974,14.401148,10.846541
2,cl2200.edf,154,1,2,0,en-universal-train.conllu 225,3,says,166,212,...,11.569777,0.0,12.469435,6.777042,9.175594,11.181264,4.0,11.569777,8.793505,9.727474
3,cl2200.edf,154,1,2,0,en-universal-train.conllu 225,4,39-year-old,212,328,...,inf,1.0,inf,10.737285,10.572124,16.052859,11.0,0.000000,14.428168,11.999637
4,cl2200.edf,154,1,2,0,en-universal-train.conllu 225,5,Pete,328,373,...,14.635095,0.0,15.891969,17.035008,11.245008,12.166563,4.0,14.635095,8.873432,7.875414
5,cl2200.edf,154,1,2,0,en-universal-train.conllu 225,6,"Broberg,",373,462,...,inf,0.0,24.246034,23.373858,21.601776,23.927165,7.0,0.000000,16.450142,16.957155
6,cl2200.edf,154,1,2,0,en-universal-train.conllu 225,7,working,462,547,...,11.926274,0.0,11.959387,12.165140,12.466643,10.604971,7.0,11.926274,7.638732,7.619304
7,cl2200.edf,154,1,2,0,en-universal-train.conllu 225,8,in,547,571,...,6.640238,0.0,6.117633,4.195356,5.082516,2.558746,2.0,6.640238,1.802303,2.136186
8,cl2200.edf,154,1,2,0,en-universal-train.conllu 225,9,the,571,605,...,5.048944,0.0,4.667835,2.086673,2.231350,2.110938,3.0,5.048944,1.591023,1.070809
9,cl2200.edf,154,1,2,0,en-universal-train.conllu 225,10,midday,605,681,...,20.174992,0.0,19.091160,16.201626,17.328781,13.978954,6.0,20.174992,11.752504,14.903147


In [ ]:
# Fix formatting issues:
from tqdm import tqdm
out_file = "surp_data/merge_output4.tsv"
with open('surp_data/merge_output3.tsv', 'r') as file:
    for i, line in tqdm(enumerate(file)):
        if i == 0:
            with open(out_file, 'w') as f:
                new_line = "DATA_FILE\tlist\tdataset_version\ttrial\tshared_text\tsentenceid\tIA_ID\tIA_LABEL\tIA_LEFT\tIA_RIGHT\tIA_FIRST_FIXATION_X\tIA_DWELL_TIME\tIA_DWELL_TIME_%\tIA_FIRST_FIXATION_DURATION\tIA_FIRST_FIXATION_INDEX\tIA_FIRST_FIXATION_PREVIOUS_FIX_IA\tIA_FIRST_FIXATION_PREVIOUS_IAREAS\tIA_FIRST_FIXATION_VISITED_IA_COUNT\tIA_FIRST_FIXATION_RUN_INDEX\tIA_FIRST_FIX_PROGRESSIVE\tIA_FIRST_SACCADE_ANGLE\tIA_FIRST_RUN_LANDING_POSITION\tIA_FIRST_RUN_DWELL_TIME\tIA_FIRST_RUN_FIXATION_COUNT\tIA_FIRST_RUN_LAUNCH_SITE\tIA_REGRESSION_PATH_DURATION\tIA_REGRESSION_IN\tIA_REGRESSION_IN_COUNT\tIA_REGRESSION_OUT\tIA_REGRESSION_OUT_FULL\tIA_REGRESSION_OUT_COUNT\tIA_REGRESSION_OUT_FULL_COUNT\tIA_FIXATION_COUNT\tIA_RUN_COUNT\tIA_SKIP\tIP_START_TIME\tIP_END_TIME\tEYE_USED\tTRIAL_FIXATION_COUNT\tTRIAL_IA_COUNT\tTRIAL_DWELL_TIME\tcorrect_answer\tkey_pressed\tsentence\tquestion\tanswered_correctly\tWORD_NORM\tOOV_BLLIP\tFREQ_BLLIP\tOOV_SUBTLEX\tFREQ_SUBTLEX\tOOV_WEB\tFREQ_WEB\tSURP_KENLM\tSURP_LSTM\tSURP_GPT2\tWORD_LEN\tFREQ_SUBTLEX_2\tEleutherAI/pythia-70m_Surprisal\tEleutherAI/pythia-160m_Surprisal\tEleutherAI/pythia-410m_Surprisal\n"
                f.write(new_line)
        else:
            with open(out_file, 'a') as f:
                f.write(line)